In [16]:
import tensorflow as tf

In [17]:
tf.__version__

'1.13.1'

## 1.tf的数据类型
### 1.1基本概念
Tensor 在数学中是“张量”

标量，矢量/向量，张量

简单地理解
- 标量表示值
- 矢量表示一个方向
- 张量表示整个空间

一维数组是矢量
多维数组是张量, 矩阵也是张量


### 1.2  四个重要的类型
1. Variable		计算图谱中的变量   
2. Tensor		（张量）一个多维矩阵，带有很多方法
3. Graph		一个计算图谱
4. Session		（会话）用来运行一个计算图谱

### 1.3 三个重要的函数
1. Variable 是一个class
```python
tf.Variable.__init__(
    initial_value=None, @Tensor
    trainable=True,
    collections=None,
    validate_shape=True,
    caching_device=None,
    name=None,
    variable_def=None,
    dtype=None)
```
2. constant 常数，**注意小写**,他不是variable,而是一个Tensor
```python
tf.constant(value, dtype=None, shape=None, name='Const')
return: a constant @Tensor
```
3. placeholder 暂时变量
```python
tf.placeholder(dtype, shape=None, name=None)
return: 一个还尚未存在的 @Tensor
```


# 2.让我们用计算图谱来实现一些简单的函数
## 2.1 四则运算

In [18]:
x = tf.Variable(3, name="x")
y = tf.Variable(4, name ="y")
f = x * x * y + y

Instructions for updating:
Colocations handled automatically by placer.


In [19]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

In [20]:
result

40

In [23]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = sess.run(f) #    和这个写发作用完全一样 result = f.eval()
result

40

## 2.1 运算的值

In [24]:
y

<tf.Variable 'y:0' shape=() dtype=int32_ref>

In [25]:
type(y)

tensorflow.python.ops.variables.RefVariable

这里没有开启session\初始化\计算，所以：  
我们看到这里他是一个Variable,还未进行运算，没有结果。

那么我们再看看constant是什么：

In [34]:
c = tf.constant(5)
print(c)
print(type(c))

Tensor("Const_4:0", shape=(), dtype=int32)
<class 'tensorflow.python.framework.ops.Tensor'>


## 2.2 Graph 操作

我们新建了一个计算图谱，把他li**临时**设定为tf的默认计算图谱，在这个图谱里初始化了俩参数

In [52]:
graph = tf.Graph()

with graph.as_default():
    value1 = tf.constant([1, 2])
    value2 = tf.Variable([3, 4])
    mul = value1 / value2

看到没,出了自定义的图空间就不是默认图了

In [61]:
value1.graph is tf.get_default_graph()

False

在开启会话的时候传入了这个图谱，那么这个session就只在Graph作用范围内进行计算

In [54]:
with tf.Session(graph=graph) as mySess:
    tf.initialize_all_variables().run()
    print(x)
#     print(mySess.run(y1)) #异常，不能参加运算
    print('对应的除法(value1, value2) = ', mySess.run(mul))
    print('对应的除法(value1, value2) = ', mul.eval())

<tf.Variable 'Variable_2:0' shape=() dtype=int32_ref>
对应的除法(value1, value2) =  [0.33333333 0.5       ]
对应的除法(value1, value2) =  [0.33333333 0.5       ]


In [58]:
with graph.as_default():
    x = tf.Variable(1)
    y1 = x + 1
    value1 = tf.constant([1, 2])
    value2 = tf.Variable([3, 4])
    mul = value1 / value2


In [59]:
with tf.Session(graph=graph) as mySess:
    tf.initialize_all_variables().run()
#     print(x)
    print(mySess.run(y1)) #ok
#     print('对应的除法(value1, value2) = ', mySess.run(mul))
#     print('对应的除法(value1, value2) = ', mul.eval())

2


## 2.3  placeholder来节省内存

In [60]:
# 10 GB数据
def load_from_remote():
    return [-x for x in range(1000)]


# 自定义的 Iterator
# yield， generator function
def load_partial(value, step):
    index = 0
    while index < len(value):
        yield value[index:index + step]
        index += step
    return


graph = tf.Graph()

with graph.as_default():
    value1 = tf.placeholder(dtype=tf.float64)
    value2 = tf.Variable([3, 4], dtype=tf.float64)
    mul = value1 * value2

with tf.Session(graph=graph) as mySess:
    tf.initialize_all_variables().run()
    # 我们想象一下这个数据是从远程加载进来的
    # 文件，网络。。。。and so on 
   
    value = load_from_remote()
    for partialValue in load_partial(value, 2):
        runResult = mySess.run(mul, feed_dict={value1: partialValue})
        # evalResult = mul.eval(feed_dict={value1: partialValue})
        print('乘法(value1, value2) = ', runResult)

乘法(value1, value2) =  [ 0. -4.]
乘法(value1, value2) =  [ -6. -12.]
乘法(value1, value2) =  [-12. -20.]
乘法(value1, value2) =  [-18. -28.]
乘法(value1, value2) =  [-24. -36.]
乘法(value1, value2) =  [-30. -44.]
乘法(value1, value2) =  [-36. -52.]
乘法(value1, value2) =  [-42. -60.]
乘法(value1, value2) =  [-48. -68.]
乘法(value1, value2) =  [-54. -76.]
乘法(value1, value2) =  [-60. -84.]
乘法(value1, value2) =  [-66. -92.]
乘法(value1, value2) =  [ -72. -100.]
乘法(value1, value2) =  [ -78. -108.]
乘法(value1, value2) =  [ -84. -116.]
乘法(value1, value2) =  [ -90. -124.]
乘法(value1, value2) =  [ -96. -132.]
乘法(value1, value2) =  [-102. -140.]
乘法(value1, value2) =  [-108. -148.]
乘法(value1, value2) =  [-114. -156.]
乘法(value1, value2) =  [-120. -164.]
乘法(value1, value2) =  [-126. -172.]
乘法(value1, value2) =  [-132. -180.]
乘法(value1, value2) =  [-138. -188.]
乘法(value1, value2) =  [-144. -196.]
乘法(value1, value2) =  [-150. -204.]
乘法(value1, value2) =  [-156. -212.]
乘法(value1, value2) =  [-162. -220.]
乘法(value1, value2)

乘法(value1, value2) =  [-1716. -2292.]
乘法(value1, value2) =  [-1722. -2300.]
乘法(value1, value2) =  [-1728. -2308.]
乘法(value1, value2) =  [-1734. -2316.]
乘法(value1, value2) =  [-1740. -2324.]
乘法(value1, value2) =  [-1746. -2332.]
乘法(value1, value2) =  [-1752. -2340.]
乘法(value1, value2) =  [-1758. -2348.]
乘法(value1, value2) =  [-1764. -2356.]
乘法(value1, value2) =  [-1770. -2364.]
乘法(value1, value2) =  [-1776. -2372.]
乘法(value1, value2) =  [-1782. -2380.]
乘法(value1, value2) =  [-1788. -2388.]
乘法(value1, value2) =  [-1794. -2396.]
乘法(value1, value2) =  [-1800. -2404.]
乘法(value1, value2) =  [-1806. -2412.]
乘法(value1, value2) =  [-1812. -2420.]
乘法(value1, value2) =  [-1818. -2428.]
乘法(value1, value2) =  [-1824. -2436.]
乘法(value1, value2) =  [-1830. -2444.]
乘法(value1, value2) =  [-1836. -2452.]
乘法(value1, value2) =  [-1842. -2460.]
乘法(value1, value2) =  [-1848. -2468.]
乘法(value1, value2) =  [-1854. -2476.]
乘法(value1, value2) =  [-1860. -2484.]
乘法(value1, value2) =  [-1866. -2492.]
乘法(value1, v